In [24]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

PREFIX='a video of '

In [25]:
events = [
        "Riding",
        "Fighting",
        "Playing",
        "Running",
        'Lying',
        "Chasing",
        "Jumping",
        "Falling",
        "Guiding",
        "Stealing",
        "Littering",
        "Tripping",
        "Pickpockering",
    ]
description = [
        "a person riding a bicycle",
        "a certain number of persons fighting",
        "a group of persons playing",
        "a person running",
        "a person lying in the floor",
        "a person chasing other person",
        "a person jumping",
        "a person falling",
        "a person guiding other person",
        "a person stealing other person",
        "a person throwing trash in the floor",
        "a person tripping",
        "a person stealing other person's pocket",
    ]

In [26]:
description=[PREFIX+desc for desc in description]
events_description = {event: desc for event, desc in zip(events, description)}

In [27]:
rute="/home/ubuntu/Database/ALL/Videos"
rute_stored='/home/ubuntu/Tesis/Storage/M4Multiclass/'

modes=[3]
usage='CLIP'
df= pd.DataFrame(columns=['Name', 'Event', 'Mode', 'TP', 'FP', 'FN', 'TN'])
for mode in modes:
    for folder_number in range(len(events)):
        folder=f"{rute}/{events[folder_number]}/"
        files= os.listdir(folder)
        for file in files:
            stored_file=file.split('.')[0]
            desc=events_description[events[folder_number]]
            try:
                info_stored=np.load(f"{rute_stored}{stored_file}_{usage}_{mode}_{desc}.npy", allow_pickle=True)
                frames_number=info_stored[0]
                predicted_events=info_stored[1]
                prompts=info_stored[2]
                event= events_description[events[folder_number]]
                #tp, fp,fn, tn=check_precision(frames_number, stored_file, predicted_events, event, description, prompts, mode)
                row = {
                'Name':stored_file, 'Event':event, 'Mode':mode, 'TP':0, 'FP':0, 'FN':0, 'TN':0
                }
                df =pd.concat([df, pd.DataFrame([row])], ignore_index=True)
            except:
                pass
print(df)
df.to_csv(f"{rute_stored}/DFS/ResultsM4.csv", index=False)

        Name                                 Event Mode TP FP FN TN
0     000350  a video of a person riding a bicycle    3  0  0  0  0
1     000289  a video of a person riding a bicycle    3  0  0  0  0
2    4_076_1  a video of a person riding a bicycle    3  0  0  0  0
3     000326  a video of a person riding a bicycle    3  0  0  0  0
4     000313  a video of a person riding a bicycle    3  0  0  0  0
..       ...                                   ...  ... .. .. .. ..
115   000304           a video of a person running    3  0  0  0  0
116   000288           a video of a person running    3  0  0  0  0
117  4_104_1           a video of a person running    3  0  0  0  0
118   000228           a video of a person running    3  0  0  0  0
119  D158_02           a video of a person running    3  0  0  0  0

[120 rows x 7 columns]


In [28]:
df_fps= pd.read_csv("/home/ubuntu/Tesis/Results/TestingDevM4MC.csv")
print(df_fps["Process time"].sum()/(60*60), "hours\n\n")
df_fps["Process time"] = df_fps["Process time"] / df_fps["Duration"]
df_fps.loc[df_fps["Name"].str.contains("_1.mp4"), "Process time"] = (
        30.0 / df_fps["Process time"]
    )
df_fps.loc[~df_fps["Name"].str.contains("_1.mp4"), "Process time"] = (
        25.0 / df_fps["Process time"]
    )
print(df_fps)

9.466602014965481 hours


            Name  Mode  True Positive  False Positive  False Negative  \
0     000350.avi     3              0               0               0   
1     000289.avi     3              0               0               0   
2    4_076_1.mp4     3              0               0               0   
3     000326.avi     3              0               0               0   
4     000313.avi     3              0               0               0   
..           ...   ...            ...             ...             ...   
115   000304.avi     3              0               0               0   
116   000288.avi     3              0               0               0   
117  4_104_1.mp4     3              0               0               0   
118   000228.avi     3              0               0               0   
119  D158_02.avi     3              0               0               0   

     True Negative                            True Event  Check event  \
0                0  a vi

In [29]:
categories = df_fps["True Event"].unique()
category_dfs = {category: df_fps[df_fps["True Event"] == category] for category in categories}
print(categories)
mAP_process = []
for i in range(len(categories)):
    df1 = category_dfs[categories[i]]
    grouped = df1.groupby("Mode")
    mean_values = grouped[["Process time"]].mean()
    mean_values = mean_values[["Process time"]]
    mAP_process.append(mean_values)
# Calculate the mean Average Precision (mAP) for each mode
mAP_values = pd.concat(mAP_process).groupby(level=0).mean()
print("\n\n", mAP_values)

['a video of a person riding a bicycle'
 'a video of a certain number of persons fighting'
 'a video of a group of persons playing' 'a video of a person running']


       Process time
Mode              
3         4.775275
